In [1]:
import os
import numpy as np
import dask
import dask.dataframe as dd
import pandas as pd

In [33]:
os.getcwd()

'C:\\Users\\A.Ram\\Desktop\\BI_Work'

In [2]:
df = dd.read_csv(os.path.join(os.getcwd(), '2019*.csv'))


In [3]:
df

,MATERIAL,PLNT,BATCH,UNRESTRICTED,TRANSIT/TRANSF.,IN QUALITY INSP.,RESTRICTED-USE,BLOCKED,RETURNS,STATUS_DATE
npartitions=21,,,,,,,,,,
,object,object,object,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [4]:
df.columns


Index(['MATERIAL', 'PLNT', 'BATCH', 'UNRESTRICTED', 'TRANSIT/TRANSF.',
       'IN QUALITY INSP.', 'RESTRICTED-USE', 'BLOCKED', 'RETURNS',
       'STATUS_DATE'],
      dtype='object')

In [5]:
df = df.rename(columns={'UNRESTRICTED' : "st1",
                       "TRANSIT/TRANSF." : "st2",
                       "MATERIAL" : 'sku_code',
                       "PLNT" : "sc_node_code",
                       "STATUS_DATE" : "status_date",
                       "BATCH" : "batch"})

In [6]:
df['st3'] = df['IN QUALITY INSP.'].astype(float) + df['RESTRICTED-USE'].astype(float)\
            + df['BLOCKED'].astype(float) + df['RETURNS'].astype(float)

In [7]:
df = df.drop(['IN QUALITY INSP.','RESTRICTED-USE','BLOCKED','RETURNS'],axis=1)


In [8]:
df.head()


,sku_code,sc_node_code,batch,st1,st2,status_date,st3
0,RL10J0AS10AH1,1002,LIAW,25.0,0.0,2019-06-01,0.0
1,RL10J0AS10AH1,1002,LIAW,1209.0,0.0,2019-06-01,0.0
2,RL10J0AS10AH1,ASR1,NaN,0.0,67.0,2019-06-01,0.0
3,RL10J0AS10AH1,GJ01,LIAW,3.0,0.0,2019-06-01,0.0
4,RL10J0AS10AH1,GJ02,LIAW,4.0,0.0,2019-06-01,0.0


In [9]:
df['sales_channel_code'] = np.nan
df['name_length']  = df['sku_code'].str.len()
df['character_check'] = df['sku_code'].astype(str).str[-1].str.isalpha()
df['BATCH_NEW'] = df['batch'].astype(str).str[-2:]
df['MATERIAL_Check'] = df['sku_code'].astype(str).str[:2]


In [10]:
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="AW") ,"REP")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="KZ") ,"REP")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="CC") ,"REP")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="DF") ,"OEM")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="ST") ,"OEM")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="OE") ,"OEM")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.BATCH_NEW=="EX") ,"REP")
df['sales_channel_code'] = df['sales_channel_code'].mask((df.name_length==13)&(df.character_check==True) ,"REP")

# df['NewCol'] = df['NewCol'].mask(new_col == 0b100, -1)
# df.loc[(df.BATCH_NEW=="AW") , 'sales_channel_code']="REP"
# df.loc[(df.BATCH_NEW=="KZ") , 'sales_channel_code']="REP"
# df.loc[(df.BATCH_NEW=="CC") , 'sales_channel_code']="REP"
# df.loc[(df.BATCH_NEW=="DF") , 'sales_channel_code']="OEM"
# df.loc[(df.BATCH_NEW=="ST") , 'sales_channel_code']="OEM"
# df.loc[(df.BATCH_NEW=="OE") , 'sales_channel_code']="OEM"
# df.loc[(df.BATCH_NEW=="EX") , 'sales_channel_code']="EXP"
# df.loc[(df.name_length==13)&(df.character_check==True) , 'sales_channel_code']="EXP"
# df.dropna(subset=['sales_channel_code'],inplace=True)


In [11]:
df_unpivoted =  df.map_partitions(pd.melt, id_vars=['status_date', 'sku_code','sc_node_code','sales_channel_code'],
                                  var_name='stock_type', value_name='stock_qty')

In [12]:
df_unpivoted = df_unpivoted.rename(columns={'status_date':'for_date'})


In [13]:
df_unpivoted


,for_date,sku_code,sc_node_code,sales_channel_code,stock_type,stock_qty
npartitions=21,,,,,,
,object,object,object,object,object,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [14]:
forecast_dd = dd.read_csv(os.path.join(os.getcwd(), 'forecast_june.csv'),dtype={'forecast_qty':float})


In [15]:
forecast_dd


,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
npartitions=1,,,,,,,,,,
,int64,int64,int64,int64,int64,object,object,object,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [16]:
dbDetails = {
    "driver": "postgresql",
    "username": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "alpha_apollo"
}

In [17]:
uri = '%(driver)s://%(username)s:%(password)s@%(host)s:%(port)s/%(database)s' % dbDetails


In [18]:
calendar_dd = dd.read_sql_table('calendar_master',
                                  uri,
                                 schema='public',
                                  index_col='for_date',
                                  divisions=['2019-06-01','2019-06-07','2019-06-14','2019-06-21','2019-06-30'],
                               ).reset_index()

In [19]:
calendar_dd

,for_date,for_month,for_year,for_day,for_day_reverse,days_in_month,week_in_month,days_week_in_month,week_in_year,days_week_in_year,day_in_week,fiscal_year_1,fiscal_year_2,quarter,fiscal_month,for_month_2,month_year,full_month,attribute_1,attribute_2,attribute_3,week_opp,days_in_week_opp,in_scope
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,
,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,int64,int64,object,object,object,object,object,object,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [20]:
products_dd = forecast_dd[['sku_code','sc_node_code','sales_channel_code','for_month','for_year']].drop_duplicates()


In [21]:
products_dd


,sku_code,sc_node_code,sales_channel_code,for_month,for_year
npartitions=1,,,,,
,object,object,object,int64,int64
,...,...,...,...,...


In [22]:
products_dd = dd.merge(calendar_dd[['for_date','for_month','for_year','days_in_month']], 
         products_dd,
         on=['for_month', 'for_year'])

In [23]:
products_dd

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code
npartitions=4,,,,,,,
,object,int64,int64,int64,object,object,object
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [24]:
forecast_dd

,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
npartitions=1,,,,,,,,,,
,int64,int64,int64,int64,int64,object,object,object,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [25]:
products_dd = dd.merge(products_dd, 
         forecast_dd,
         on=['for_month', 'for_year', 'sku_code', 'sc_node_code','sales_channel_code'],
            how='left')

In [26]:
products_dd['forecast_dly'] = products_dd['forecast_qty']/products_dd['days_in_month']


In [27]:
final_df = products_dd.merge(df_unpivoted,
                 on=['sku_code', 'sc_node_code','sales_channel_code','for_date'],
                 how='left')

In [28]:
import dask
from dask.distributed import Client
# By default this sets up 1 worker per core
client = Client()
client.cluster

In [29]:
client

Client Scheduler: tcp://127.0.0.1:60736 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.06 GB


In [72]:
%%time
forecast_dd.compute().head()

Wall time: 1.27 s


,source_tag,for_month,for_year,in_month,in_year,sku_code,sc_node_code,sales_channel_code,forecast_qty,forecast_value
0,1,6,2019,6,2019,RL10J0AS10AH1,1002,REP,1946.0,4378500.0
1,1,6,2019,6,2019,RL10J0AS10AH1,AP01,REP,300.0,675000.0
2,1,6,2019,6,2019,RL10J0AS10AH1,AP02,REP,15.0,33750.0
3,1,6,2019,6,2019,RL10J0AS10AH1,AP03,REP,30.0,67500.0
4,1,6,2019,6,2019,RL10J0AS10AH1,AP04,REP,200.0,450000.0


In [84]:
%%time
df_forecast=products_dd.compute()

Wall time: 31.7 s


In [85]:
df_forecast.head()

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code,source_tag,in_month,in_year,forecast_qty,forecast_value,forecast_dly
0,2019-06-02,6,2019,30,RL10J0AS10AH1,1002,REP,1,6,2019,1946.0,4378500.0,64.866667
1,2019-06-02,6,2019,30,RL10J0AS10AH1,AP01,REP,1,6,2019,300.0,675000.0,10.000000
2,2019-06-02,6,2019,30,RL10J0AS10AH1,AP02,REP,1,6,2019,15.0,33750.0,0.500000
3,2019-06-02,6,2019,30,RL10J0AS10AH1,AP03,REP,1,6,2019,30.0,67500.0,1.000000
4,2019-06-02,6,2019,30,RL10J0AS10AH1,AP04,REP,1,6,2019,200.0,450000.0,6.666667


In [86]:
%%time
df_stock=df_unpivoted.compute()

Wall time: 41.7 s


In [87]:
df_stock.head()

,for_date,sku_code,sc_node_code,sales_channel_code,stock_type,stock_qty
0,2019-06-01,RL10J0AS10AH1,1002,REP,batch,LIAW
1,2019-06-01,RL10J0AS10AH1,1002,REP,batch,LIAW
2,2019-06-01,RL10J0AS10AH1,ASR1,NaN,batch,NaN
3,2019-06-01,RL10J0AS10AH1,GJ01,REP,batch,LIAW
4,2019-06-01,RL10J0AS10AH1,GJ02,REP,batch,LIAW


In [88]:
%%time
final_df_2 = df_forecast.merge(df_stock,
                 on=['sku_code', 'sc_node_code','sales_channel_code','for_date'],
                 how='left')

Wall time: 9.21 s


In [89]:
final_df_2.head()

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code,source_tag,in_month,in_year,forecast_qty,forecast_value,forecast_dly,stock_type,stock_qty
0,2019-06-02,6,2019,30,RL10J0AS10AH1,1002,REP,1,6,2019,1946.0,4378500.0,64.866667,NaN,NaN
1,2019-06-02,6,2019,30,RL10J0AS10AH1,AP01,REP,1,6,2019,300.0,675000.0,10.000000,NaN,NaN
2,2019-06-02,6,2019,30,RL10J0AS10AH1,AP02,REP,1,6,2019,15.0,33750.0,0.500000,NaN,NaN
3,2019-06-02,6,2019,30,RL10J0AS10AH1,AP03,REP,1,6,2019,30.0,67500.0,1.000000,NaN,NaN
4,2019-06-02,6,2019,30,RL10J0AS10AH1,AP04,REP,1,6,2019,200.0,450000.0,6.666667,NaN,NaN


In [30]:
%%time
df1 = final_df.compute()

Wall time: 1min 34s


In [31]:
df1.head()


,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code,source_tag,in_month,in_year,forecast_qty,forecast_value,forecast_dly,stock_type,stock_qty
0,2019-06-02,6,2019,30,RL10J0AS10AH1,BR04,REP,1,6,2019,4.0,9000.0,0.133333,NaN,NaN
1,2019-06-02,6,2019,30,RL10J0AS10AH1,CG05,REP,1,6,2019,2.0,4500.0,0.066667,NaN,NaN
2,2019-06-02,6,2019,30,RL10O0AS10AH1,CGR1,REP,1,6,2019,117.0,274950.0,3.900000,NaN,NaN
3,2019-06-02,6,2019,30,RL10O0AS10AH1,KA06,REP,1,6,2019,10.0,23500.0,0.333333,NaN,NaN
4,2019-06-02,6,2019,30,RL10O0AS10AH1,MA03,REP,1,6,2019,20.0,47000.0,0.666667,NaN,NaN


In [32]:
df1.to_parquet('stockout.parquet')

In [35]:
df1 = dd.read_parquet('stockout.parquet')


In [37]:
df1.head()

,for_date,for_month,for_year,days_in_month,sku_code,sc_node_code,sales_channel_code,source_tag,in_month,in_year,forecast_qty,forecast_value,forecast_dly,stock_type,stock_qty
0,2019-06-02,6,2019,30,RL10J0AS10AH1,BR04,REP,1,6,2019,4.0,9000.0,0.133333,None,None
1,2019-06-02,6,2019,30,RL10J0AS10AH1,CG05,REP,1,6,2019,2.0,4500.0,0.066667,None,None
2,2019-06-02,6,2019,30,RL10O0AS10AH1,CGR1,REP,1,6,2019,117.0,274950.0,3.900000,None,None
3,2019-06-02,6,2019,30,RL10O0AS10AH1,KA06,REP,1,6,2019,10.0,23500.0,0.333333,None,None
4,2019-06-02,6,2019,30,RL10O0AS10AH1,MA03,REP,1,6,2019,20.0,47000.0,0.666667,None,None


In [38]:
df1.describe()

,for_month,for_year,days_in_month,source_tag,in_month,in_year,forecast_qty,forecast_value,forecast_dly
npartitions=1,,,,,,,,,
,int64,int64,int64,int64,int64,int64,float64,float64,float64
,...,...,...,...,...,...,...,...,...


In [39]:
df1.to_csv('stockout.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\A.Ram\\Desktop\\BI_Work\\stockout.csv\\0.part'

TODO

-Read and Write to Database
-Make the system as master and VM's as slave and run the code
-Check is alphabet function and picking last two elements of columnm